In [9]:
#reading in data 
import pandas as pd
test_features = pd.read_csv('test_features.csv')
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')

In [19]:
!conda install -c conda-forge category_encoders --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - category_encoders


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    scikit-learn-0.20.1        |   py36h4989274_0         5.7 MB
    category_encoders-2.0.0    |             py_1          41 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.9 MB

The following NEW packages will be INSTALLED:

    category_encoders: 2.0.0-py_1            conda-forge

The following packages will be UPDATED:

    ca-certif

In [20]:
from numba import jit

In [21]:
import dask.dataframe as dd
from dask.distributed import Client
from dask import compute, delayed

In [22]:
#test_features = dd.read_csv('test_features.csv')
#train_features = dd.read_csv('train_features.csv')
#train_labels = dd.read_csv('train_labels.csv')

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_train_val, y_train, y_train_val  = train_test_split(train_features, train_labels,
                                                               test_size=len(test_features))

In [24]:
#since we don't have domain expertise we are going to use our intuition to create a "ML baseline' 
#the commented remarks were initial observations, the model was improved, however remarks show learning process 
def wrangle(X):
    X = X.copy()
    
    #drop some columns 
    X = X.drop(columns='id') #id is random 
    X = X.drop(columns='date_recorded') #we don't care about the recorded row date 
    X = X.drop(columns='longitude') #does not add relevant data
    X = X.drop(columns='latitude') #does not add relevant data 
    X = X.drop(columns='num_private') # data dictionary has no description for this feature
    X = X.drop(columns='public_meeting') # data dictionary provides only "true/false"
    X = X.drop(columns='extraction_type_group') # not duplicate but very similar thus contains high variance of source feature
    X = X.drop(columns='extraction_type_class') # not duplicate but very similar thus contains high variance of source feature
    X = X.drop(columns='payment_type') # duplicate feature
    X = X.drop(columns='quantity_group') # duplicate feature
    X = X.drop(columns='source_type') # not duplicate but bery similar thus contains high variance of source feature
    X = X.drop(columns='waterpoint_type_group') # not duplicate but very similar thus contains high variance of source feature
    X = X.drop(columns='region') #quasi-duplicates
    X = X.drop(columns='quality_group') #quasi-duplicates
    X = X.drop(columns='source_class') #quasi-duplicates
    X = X.drop(columns='management_group') #quasi-duplicates
  
    #drop high cardinality categoricals
    X = X.drop(columns='funder')
    X = X.drop(columns='installer')
    X = X.drop(columns='wpt_name')
    X = X.drop(columns='subvillage')
    X = X.drop(columns='ward')
    X = X.drop(columns='scheme_name')
               
    
    #drop Nans from 'scheme_management' and 'permit' feature 
    X['scheme_management'] = X['scheme_management'].dropna(axis=0, how='all')
    X['permit'].dropna(axis=0)
    
    return X 


X_train = wrangle(X_train)
X_train_val = wrangle(X_train_val)  
X_test = wrangle(test_features)

In [25]:
import category_encoders as ce
def encode(X_train, X_train_val, X_test):
    X_train = X_train.copy()
    X_train_val = X_train_val.copy()
    X_test = X_test.copy()
    
    encode = ce.OrdinalEncoder()
    X_train =  encode.fit_transform(X_train)
    X_train_val = encode.fit_transform(X_train_val)
    X_test = encode.fit_transform(X_test)
    
    return X_train, X_train_val, X_test

In [26]:
X_train, X_train_val, X_test = encode(X_train, X_train_val, X_test)
X_train.shape, X_train_val.shape, X_test.shape

((45042, 18), (14358, 18), (14358, 18))

In [27]:
y_train = y_train['status_group']
y_train_val = y_train_val['status_group']

In [28]:
X_train.head()

,amount_tsh,gps_height,basin,region_code,district_code,lga,population,recorded_by,scheme_management,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type
54187,250.0,1312,1,16,2,1,1325,1,1,1,1984,1,1,1,1,1,1,1
51196,0.0,0,2,12,3,2,0,1,2,1,0,1,2,2,1,2,2,1
15569,1000.0,552,3,10,1,3,1,1,2,2,1998,1,2,3,1,1,1,1
8156,10000.0,1419,4,3,1,4,1,1,3,1,2012,1,3,1,1,3,2,2
27172,0.0,1581,5,13,1,5,120,1,4,1,2008,2,4,2,1,1,3,1


In [36]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=None, n_estimators=300,
                                n_jobs=-1, random_state=42, verbose=1)
model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   11.8s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=42, verbose=1, warm_start=False)